In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import pickle
import pyreadr
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

# Prepare data set

In [ ]:
# read T1DGC data set
t1dgc = pd.read_csv('T1DGCAllSNPs.csv')

# remove the first unnecessary column
t1dgc = t1dgc.drop(['Unnamed: 0'], axis=1)

# drop unnecessary columns for Data pre-processing
data = t1dgc[t1dgc.columns.drop(['sample.id', 'prediction', 'sex', 'HLA.DQB1_allele1', 'HLA.DQB1_allele2', 'DQB1', 'HLA.DRB1_allele1',
       'HLA.DRB1_allele2', 'DRB1'])]

# binarize labels column (affected: negative = 0, positive  = 1)
lb = preprocessing.LabelBinarizer()
labels = lb.fit_transform(list(data.affected))

# drop affected column, since we have already binarized labels
data = data[data.columns.drop(['affected'])]


# insert binarized labels as final column in our data set
data.insert(len(data.columns), 'targets', labels, True) 

# separate features and target label in data set
t1dgsData_AllSNPs_HIBAG_SNP_data = {'data' : data.loc[:, data.columns != 'targets'].values, 'target' : data['targets'].values}

# save all SNP file
with open('Data_AllSNPs_HIBAG_SNPS.pickle', 'wb') as f:
    pickle.dump([t1dgsData_AllSNPs_HIBAG_SNP_data,data], f)
    
# with open('Data_AllSNPs_HIBAG_SNPS.pickle', 'rb') as f:
#     t1dgsData_AllSNPs_HIBAG_SNP_data,data = pickle.load(f)

# Logistic regression with Nested cross-validation

Nested cross-validation allows us to measure uncertainty and variance of model. It is more generalisable than traditional 
cross-validation. Nested CV composed of 10 outer and 10 inner folds. For every outer fold, fold is divided train / test set and training set is used with 10-fold traditional cross validation for hyperparameter tuning, and test set for performance assessment. Finally at the end of NestedCV there are 10 different models which we can measure variance of performance. 

## Train 10x10 Nested CV

In [ ]:
# Load data set
with open('Data_AllSNPs_HIBAG_SNPS.pickle', 'rb') as f:
    t1dgsData_AllSNPs_HIBAG_SNP_data,data = pickle.load(f)

# retrieve data and target labels
X = t1dgsData_AllSNPs_HIBAG_SNP_data['data']
y = t1dgsData_AllSNPs_HIBAG_SNP_data['target']



# 10 Outer Loops
split = 10
cv_outer = StratifiedKFold(n_splits=split, shuffle = True, random_state = 42)
cv_outer = cv_outer.split(X,y)
train_list = []
test_list = []
for train,test in cv_outer:
    train_list.append(train)
    test_list.append(test)
    
    
# create a pipeline with feature scaling, then Logistic Regression
pipe_sgd = Pipeline([('scl', StandardScaler()),
                    ('clf', LogisticRegression(solver = 'saga', penalty = 'l1', max_iter=10000,multi_class = 'auto', n_jobs = -1))])

# Create parameters dictionary to look for optimal hyperparameters
param_dist_sgd = {'clf__C': np.logspace(-5,2,10)}


# Save train / test AUC scores and also selected feature coefficients for further analysis
mean_fpr = np.linspace(0,1,100)
tprs = list()
Optims_list = list()
test_AUCs = list()
train_AUCS = list()
test_roc = list()
features_list = list()

# Loop for every outer fold
for i in range(split):
    print("In progress: ", i+1,"/",split)
    
    # Inner loop: initialize 10-fold cross-validation
    cv = StratifiedKFold(n_splits=split, shuffle = True, random_state = 42)
    
    
    # Construct GreedSearch with 10-folds cross-validation
    sgd_randomized_pipe = GridSearchCV(estimator = pipe_sgd,param_grid=param_dist_sgd, 
                                       cv=cv, n_jobs=11, scoring = 'roc_auc', verbose = 3)
    

    # Train pipeline on GridSearch for Train data set
    sgd_randomized_pipe.fit(X[train_list[i]], y[train_list[i]])
    

    # best optimal parameters
    Optims = sgd_randomized_pipe.cv_results_['params'][sgd_randomized_pipe.best_index_]
    Optims_list.append(Optims)
    
    
    # Construct pipeline with optimal parameters
    pipe_lr = Pipeline([('scl', StandardScaler()), ('clf', LogisticRegression(C = Optims['clf__C'], solver = 'saga', 
                                                                              penalty = 'l1', max_iter=10000, multi_class = 'auto', n_jobs = 11))])

    # Train pipeline on Train data
    pipe_lr.fit(X[train_list[i]], y[train_list[i]])
    
    
    # Train AUC 
    y_probs_train = pipe_lr.predict_proba(X[train_list[i]])[:,1]
    fp_rate_train, tp_rate_train, thresholds = roc_curve(y[train_list[i]], y_probs_train)
    train_auc = auc(fp_rate_train, tp_rate_train)
    train_AUCS.append(train_auc)
    
    
    # predict test score
    y_probs_test = pipe_lr.predict_proba(X[test_list[i]])[:,1]
    fp_rate_test, tp_rate_test, thresholds = roc_curve(y[test_list[i]], y_probs_test)
    tprs.append(np.interp(mean_fpr, fp_rate_test, tp_rate_test))
    
    # store roc_aucs for plotting ROC curves later
    roc_aucs = [fp_rate_test,tp_rate_test]
    test_roc.append(roc_aucs)
    
    # test AUC 
    test_auc = auc(fp_rate_test, tp_rate_test)
    test_AUCs.append(test_auc)
    
    
    # Store selected features for further heatmap plotting
    coef = pipe_lr.named_steps.clf.coef_
    features = list(data.loc[:, data.columns != 'targets'].columns.values)
    data_coeff = {'coefficient': list(abs(coef[0])), 'features': features}
    data_coeff = pd.DataFrame(data=data_coeff)
    data_coeff.sort_values(by=['coefficient'], inplace=True, ascending=False)
    data_coeff = data_coeff[data_coeff['coefficient']!= 0]
    features_list.append(data_coeff)
    
    
# store variables for further usecase
with open('AllSNPs_HIBAG_NestedCV.pickle', 'wb') as f:
    pickle.dump([mean_fpr, tprs, Optims_list, train_AUCS, test_AUCs, test_roc, features_list], f)
    
# with open('AllSNPs_HIBAG_NestedCV.pickle', 'rb') as f:
#     mean_fpr, tprs, Optims_list, train_AUCS, test_AUCs, test_roc, features_list = pickle.load(f)

## ROC-AUC curves

In [ ]:
# plot ROC curves for 10 different models, show variance / mean AUC and performance AUC for every model

fig1 = plt.figure(figsize=[8,8])
# Plot 45 degree line
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')

for i in range(split):
    plt.plot(test_roc[i][0], test_roc[i][1],label=r'Model: %d,(AUC = %0.2f )' % (i+1,test_AUCs[i]),lw=2)

# compute m-str/m+std
ci = np.std(tprs, axis = 0)

# Compute mean of true positives and false positives
mean_tpr = np.mean(tprs, axis=0)

plt.fill_between(mean_fpr, (mean_tpr-3*ci), (mean_tpr+3*ci),color='blue', alpha=.1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves for Logistic Regression: AUCs std: %0.2f, AUCs mean: %0.2f' % (np.std(test_AUCs),np.mean(test_AUCs)))
plt.legend(loc="lower right")
plt.grid()
plt.show()

## Coefficient plots for selected features from Nested CV

In [ ]:
# combine selected features for every model from Nested CV

df = pd.merge(features_list[0],features_list[1],how = 'outer', on = 'features')

for i in range(2,split):
    df = pd.merge(df, features_list[i],how = 'outer', on = 'features')

df.index = df['features'].values
df = df.drop(['features'], axis=1)
columns = list()
for i in range(split):
    model = 'model_{}'.format(str(i+1))
    columns.append(model)
df.columns = columns

# since number of selected features might not be the same for every model
# outer join will show NaNs for some features in specific models. 
# So, fill zero for NaN and round coefficients until 2 digits.

df = df.fillna(0)
df = df.round(2)
df = df.T

# plot heat map feature coefficients from Nested CV
fig = plt.figure(figsize=[30,8])
chart = sns.heatmap(df, annot=False)
chart.set_title('Heatmap for the feature coefficients', fontsize=25)
chart.set_ylabel('Best models from NestedCV', fontsize=20)
chart.set_xlabel('Features', fontsize=20)
y = chart.set_yticklabels(chart.get_yticklabels(), rotation=0)
y = chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

## AUC vs Top features for every model in Nested CV

AUC vs Top ranked features allows us to observe how every model behaves in different parts of dataset

In [ ]:
# load data
with open('Data_AllSNPs_HIBAG_SNPS.pickle', 'rb') as f:
    t1dgsData_AllSNPs_HIBAG_SNP_data,data_old = pickle.load(f)

# initialize lists to store models performances
models = list()
intervals = list()

# for every test set of outer folds, run respective best model of that fols on 
# by adding the top features one by one
for i in range(split):
    print("In progress: {} / {}".format(i+1,split))
    
    # retrieve Optims
    Optims = Optims_list[i]

    # top Features from Lasso
    data_coeff = features_list[i]
    top_ranked_features = list(data_coeff[data_coeff['coefficient'] != 0].features.values)

    labels = data_old['targets'].values

    # Select data frame with top ranked features
    data = pd.DataFrame(data_old, columns = top_ranked_features)

    # insert binarized labels as final column in our data set
    data.insert(len(data.columns), 'targets', labels, True) 

    # separate features and target label in data set
    t1dgsData = {'data' : data.loc[:, data.columns != 'targets'].values, 'target' : data['targets'].values}

    X = t1dgsData['data']
    y = t1dgsData['target']
    X_train = X[train_list[i]]
    y_train = y[train_list[i]]
    X_test = X[test_list[i]]
    y_test = y[test_list[i]]
    
    
    # loop for every top-ranked feature and train it on train set of respective fold and 
    # measure performance on test set
    n_features = X_train.shape[1]
    testAuc = list()
    for feature in range(n_features):
        
        # create a pipeline with optimal parameters
        pipe_lr = Pipeline([('scl', StandardScaler()), ('clf', LogisticRegression(C = Optims['clf__C'], solver = 'saga', 
                                                                              penalty = 'l1', max_iter=10000, multi_class = 'auto', n_jobs = 11))])
        # Train pipeline on Train data
        pipe_lr.fit(X_train[:,:feature+1], y_train)

        # Test AUC on trained model
        y_probs_test = pipe_lr.predict_proba(X_test[:,:feature+1])[:,1]
        fp_rate_test, tp_rate_test, thresholds = roc_curve(y_test, y_probs_test)
        test_auc = auc(fp_rate_test, tp_rate_test)
        testAuc.append(test_auc)
    
    models.append(testAuc)
    x_axis = np.linspace(1, X_train.shape[1]+1, X_train.shape[1])
    intervals.append(x_axis)
    
# store variables for further usecase
with open('AllSNPs_HIBAG_AUCvsTopFeatures_NestedCV.pickle', 'wb') as f:
    pickle.dump([intervals, models], f)
    
# with open('AllSNPs_HIBAG_AUCvsTopFeatures_NestedCV.pickle', 'rb') as f:
#     intervals, models = pickle.load(f)

In [ ]:
# plot AUCs vs Top-ranked features for every model from Nested CV
fig3 = plt.figure(figsize=[8,8])
for i in range(split):
    plt.plot(intervals[i],models[i],label="Model: %d" % (i+1),lw=2)

plt.grid()
plt.title("NestedCV: Top ranked features vs AUC" )
plt.ylabel("AUC")
plt.xlabel("Top Ranked Features")
plt.legend(loc="lower right")